In [2]:
from datasets import Dataset
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer
import torch
import os

In [43]:
df = pd.read_csv("/home/silxxor/Projects/Addressee_detector/data.csv")
dataset = Dataset.from_pandas(df[['text', 'label']])
dataset = dataset.train_test_split(test_size=0.2, seed=42)
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
def tokenize(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)
dataset = dataset.map(tokenize, batched=True)

model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

from transformers.utils import logging
logging.set_verbosity_info()

training_args = TrainingArguments(
    output_dir="models/addressee_detector",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_strategy="epoch",
    weight_decay=0.01,
    disable_tqdm=False,  # Make sure it's not disabled
    learning_rate=2e-5,
    use_cpu=False
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

from transformers import TrainerCallback

from time import time

start_time = time()


class EpochTimeCallback(TrainerCallback):
    def __init__(self):
        self.epoch_start = None
    
    def on_epoch_begin(self, args, state, control, **kwargs):
        self.epoch_start = time()
    
    def on_epoch_end(self, args, state, control, **kwargs):
        epoch_time = time() - self.epoch_start
        total_elapsed = time() - start_time
        epochs_remaining = args.num_train_epochs - state.epoch
        estimated_remaining = epoch_time * epochs_remaining
        print(f"\n{'='*50}")
        print(f"Epoch {int(state.epoch)} completed in {epoch_time/60:.2f} minutes")
        print(f"Total elapsed: {total_elapsed/60:.2f} minutes")
        print(f"Estimated remaining: {estimated_remaining/60:.2f} minutes")
        print(f"{'='*50}\n")
        
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics,
    callbacks=[EpochTimeCallback()]
)

trainer.train()

loading configuration file config.json from cache at /home/silxxor/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.53.3",
  "vocab_size": 30522
}

loading file vocab.txt from cache at /home/silxxor/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/vocab.txt
loading file tokenizer.json from cache at /home/silxxor/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f7

Map:   0%|          | 0/1082 [00:00<?, ? examples/s]

Map:   0%|          | 0/271 [00:00<?, ? examples/s]

loading configuration file config.json from cache at /home/silxxor/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.53.3",
  "vocab_size": 30522
}

loading weights file model.safetensors from cache at /home/silxxor/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/model.safetensors
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequen

Step,Training Loss


...


Epoch 2 completed in 0.39 minutes
Total elapsed: 0.86 minutes
Estimated remaining: 0.39 minutes



Model weights saved in models/addressee_detector/checkpoint-272/model.safetensors
Saving model checkpoint to models/addressee_detector/checkpoint-408
Configuration saved in models/addressee_detector/checkpoint-408/config.json



Epoch 3 completed in 0.40 minutes
Total elapsed: 1.32 minutes
Estimated remaining: 0.00 minutes



Model weights saved in models/addressee_detector/checkpoint-408/model.safetensors


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=408, training_loss=0.1836420507992015, metrics={'train_runtime': 83.1556, 'train_samples_per_second': 39.035, 'train_steps_per_second': 4.906, 'total_flos': 107497294009344.0, 'train_loss': 0.1836420507992015, 'epoch': 3.0})

# Test

In [44]:
# Load best model
from transformers import pipeline

classifier = pipeline("text-classification", model="models/addressee_detector/checkpoint-408", tokenizer=tokenizer)

# Test
test_cases = [
    # Basic sanity checks
    "yeah this coffee sucks",
    "Lucy what time is it",
    "I'm tired",
    "hey Lucy help me",
    "fuck this code",
    "fuck this code Lucy",
    "hey man how are you doing?",
    "yo Lucy",
    "hey man", 
    "you know",  
    "I think you should",  
    "what the fuck",
    
    # Name variations (not in training)
    "hey assistant what's up",
    "assistant can you help",
    "okay assistant",
    "computer help me",
    "hey computer",
    
    # Ambiguous addressing
    "you should see this",
    "we need to fix this",
    "someone help me",
    "anybody know why",
    "can anyone explain",
    
    # Questions without clear addressing
    "where did I put that",
    "when was the last time",
    "who broke this",
    "which one is better",
    "how come this doesn't work",
    
    # Statements that could be addressed
    "you won't believe this",
    "we're gonna need more time",
    "there's a problem here",
    "something's not right",
    "everything's broken",
    
    # Imperatives (commands)
    "show me the logs",
    "check the database",
    "run the tests again",
    "stop the server",
    "restart everything",
    
    # Self-directed questions
    "why did I do it this way",
    "what was I thinking",
    "how did this ever work",
    "where was I going with this",
    "when did this break",
    
    # Incomplete/fragmented
    "so like the thing with",
    "yeah but the problem is",
    "okay so basically",
    "I mean the issue here",
    "well obviously we can't",
    
    # Multiple interpretations
    "you know what I'm saying",
    "can't believe this shit",
    "gotta be kidding me",
    "make it stop",
    "fix this mess",
    
    # Conversational but not addressed
    "anyway moving on",
    "whatever I'll deal with it",
    "fine I guess that works",
    "alright then",
    "cool story",

    # Talking to another person in the room
    "she can understand if i walk to her or not",
    "yeah she is very smart",
    "and she is funny too",
    "she is awesome",
    "yeah she is the best so far",
]

for text in test_cases:
    result = classifier(text)
    print(f"{text:30} -> {result[0]['label']} ({result[0]['score']:.3f})")

loading configuration file models/addressee_detector/checkpoint-408/config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.53.3",
  "vocab_size": 30522
}

loading configuration file models/addressee_detector/checkpoint-408/config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "ma

yeah this coffee sucks         -> LABEL_0 (0.996)
Lucy what time is it           -> LABEL_1 (0.997)
I'm tired                      -> LABEL_0 (0.998)
hey Lucy help me               -> LABEL_1 (0.996)
fuck this code                 -> LABEL_0 (0.991)
fuck this code Lucy            -> LABEL_1 (0.996)
hey man how are you doing?     -> LABEL_1 (0.879)
yo Lucy                        -> LABEL_1 (0.996)
hey man                        -> LABEL_0 (0.856)
you know                       -> LABEL_0 (0.994)
I think you should             -> LABEL_0 (0.998)
what the fuck                  -> LABEL_0 (0.957)
hey assistant what's up        -> LABEL_1 (0.995)
assistant can you help         -> LABEL_1 (0.996)
okay assistant                 -> LABEL_1 (0.995)
computer help me               -> LABEL_1 (0.996)
hey computer                   -> LABEL_1 (0.978)
you should see this            -> LABEL_0 (0.997)
we need to fix this            -> LABEL_0 (0.998)
someone help me                -> LABEL_1 (0.895)


In [1]:
from addressee_detector import AddresseeDetector
addressee_detector = AddresseeDetector()
addressee_detector.should_reply("hi Lucy", 2)

/home/silxxor/miniconda3/envs/realtimebot/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/silxxor/miniconda3/envs/realtimebot/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
Device set to use cuda:0


True